In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3'

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [3]:
import functools

def nq_dataset_fn(split, shuffle_files=False):
    del shuffle_files
    ds = tf.data.TextLineDataset(['cnn-summarization.tsv'])

    ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Map each tuple to a {"question": ... "answer": ...} dict.
    ds = ds.map(lambda *ex: dict(zip(["question", "answer"], ex)))
    return ds
    
def trivia_preprocessor(ds):
    def to_inputs_and_targets(ex):
        """Map {"question": ..., "answer": ...}->{"inputs": ..., "targets": ...}."""
        return {
            "inputs":
                 tf.strings.join(
                     ["ringkasan: ", ex["question"]]),
            "targets": ex["answer"]
        }
    return ds.map(to_inputs_and_targets, 
                    num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [6]:
t5.data.TaskRegistry.remove('cnn_summary')
t5.data.TaskRegistry.add(
    'cnn_summary',
    dataset_fn=nq_dataset_fn,
    splits=["train"],
    text_preprocessor=[trivia_preprocessor],
    sentencepiece_model_path='sp10m.cased.t5.model',
    postprocess_fn=t5.data.postprocessors.lower_text, 
    metric_fns=[t5.evaluation.metrics.accuracy],
)

In [7]:
nq_task = t5.data.TaskRegistry.get("cnn_summary")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 1024, "targets": 256})

In [8]:
for ex in tfds.as_numpy(ds.take(1)):
    print(ex)

{'inputs_plaintext': b"ringkasan: (CNN) - Edward Dolman adalah ketua pegawai eksekutif pertama Christie untuk menjadi seorang pakar, yang memahami tekanan dan sifat menilai seni. Sebahagian daripada misinya menarik perhatian pakar yang bermotivasi tinggi ke dalam perniagaan mereka. Seorang pekerja Christie berdiri di samping lukisan di rumah lelongan Christie di London pada 4 Julai. Dolman dilahirkan di London pada tahun 1960 dan menghadiri Kolej Dulwich dari 1971-1978, sebelum belajar Sejarah Seni di Southampton University. Beliau kemudiannya melengkapkan ijazah di Pusat Pengajian untuk Fine and Decorative Arts, yang mempunyai hubungan erat dengan Victoria dan Albert Museum. Tidak menjadi kelahiran darah biru, dia menyertai Christie kerana hubungannya dengan Christie'S South Kensington, bermula sebagai porter di jabatan perabot. Selepas setahun dia memegang peranan pakar dalam jabatan perabot di South Kensington, di mana dia akan melihat segala-galanya dari perabot Eropah ke China dal

In [9]:
t5.data.MixtureRegistry.remove("summary_bahasa")
t5.data.MixtureRegistry.add(
    "summary_bahasa",
    ["cnn_summary"],
     default_rate=1.0
)

In [10]:
# import gin
# gin.parse_config_file('pretrained_models_base_operative_config.gin')

In [11]:
model = t5.models.MtfModel(
    model_dir='t5-base-summary',
    tpu=None,
    tpu_topology=None,
    model_parallelism=2,
    batch_size=8,
    sequence_length={"inputs": 1024, "targets": 256},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["gpu:0"]
)

In [12]:
model.finetune(
    mixture_or_task_name="summary_bahasa",
    pretrained_model_dir='t5-base-bahasa',
    finetune_steps=1000
)

INFO:tensorflow:Using config: {'_model_dir': 't5-base-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training

In [13]:
with open('cnn-summarization.tsv') as fopen:
    data = fopen.read().split('\n')
    
data[0]

'(CNN) - Republik Sudan Selatan yang baru merdeka telah mengumumkan kematian pemimpin pemberontak terkemuka. Lt. Gen. George Athor dan salah seorang tenteranya terbunuh dalam pertempuran dengan tentera negara itu, Naib Presiden Riek Machar kepada wartawan Selasa. Dia berkata mereka telah menyusup ke Sudan Selatan dalam perjalanan keluar dari Rwanda melalui kawasan sempadan timur Republik Demokratik Congo dan Uganda. Tuntutan ini belum disahkan oleh badan bebas. Seorang komander pembangkang Tentera Pembebasan Rakyat Sudan, atau Spla, Athor memberontak setelah kalah dalam pemilihan gubernur pada musim bunga tahun 2010. Dia menyeru banyak pertempuran yang telah dirancang setelah pengumuman kekalahannya. "Di bawah spanduk Gerakan Demokratik Sudan Selatan (SSDM) dan sayap tenteranya, Tentera Sudan Selatan, Gen. Athor dianggap sebagai pemberontak pemilihan pasca pemilihan yang paling kuat" kata penyelidikan bebas yang berasaskan Geneva dalam laporannya pada tahun 2011 tentang apa kemudian Su

In [14]:
with tf.io.gfile.GFile('test.txt', "w") as f:
    f.write("ringkasan: %s\n" % data[0].split('\t')[0])

In [15]:
model.predict(
    input_file='test.txt',
    output_file='out.txt',
    temperature=1.0,
    sentencepiece_model_path='sp10m.cased.t5.model'
)

INFO:tensorflow:Using config: {'_model_dir': 't5-base-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training